# Calibração de modelos de classificação
Esse notebook mostra como podemos calibrar as previsões de modelos de classificação para que elas reflitam mais precisamente a probabilidade do evento ocorrer. Calibrar modelos de classificação é muito importante para sua correta aplicação em contextos práticos, pois facilita a sua interpretação por humanos em vários contextos. Além disso, a calibração permite o uso direto dos modelos em cálculos de negócio, como a perda esperada de crédito.

Os modelos não foram criados iguais no que se refere à calibração das previsões: modelos de árvore e Naive Bayes são notórios por gerar previsões descalibradas[[1]](https://scikit-learn.org/stable/modules/calibration.html#calibration). Usaremos as ferramentas do Scikit-Learn para resolver esse problema, usando dados simulados.

## Setup

In [1]:
# bibliotecas
import pandas as pd
import functools
from scipy import stats
from sklearn import naive_bayes, tree, calibration, pipeline, metrics, datasets, model_selection, linear_model

In [2]:
# gerando dados com dimensão (40_000, 5)
X, y = datasets.make_classification(
    n_samples=40_000,
    n_features=5,
    n_informative=5,
    n_redundant=0,
    class_sep=0.5
)
print(X.shape)
print(y.shape)

(40000, 5)
(40000,)


In [3]:
# separando os dados em treinamento e teste
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y,
    test_size=0.5,
    stratify=y,
    random_state=73
)

## Modelos

In [4]:
models = {
    "logreg": linear_model.LogisticRegression(),
    "naive_bayes": naive_bayes.GaussianNB(),
    "dtc": tree.DecisionTreeClassifier(),
}

In [5]:
# rodando os modelos
for _, m in models.items():
    m.fit(X_train, y_train)

Para medir o desempenho de modelos no que se refere à calibração de probabilidade, temos duas opções no Sci-kit Learn: o [Score de Brier](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.brier_score_loss.html#sklearn.metrics.brier_score_loss) e a [Log loss](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html#sklearn.metrics.log_loss)

In [6]:
performance = {
    model_name: {
        "brier": metrics.brier_score_loss(y_test, model.predict_proba(X_test)[:, 1]),
        "logloss": metrics.log_loss(y_test, model.predict_proba(X_test)[:, 1]),
        # para mostrar que a calibração não altera a eficácia preditiva do modelo
        "auc": metrics.roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    }
    for model_name, model in models.items()
}
performance

{'logreg': {'brier': 0.17328984236755587,
  'logloss': 0.5207789447056256,
  'auc': 0.8191846947961736},
 'naive_bayes': {'brier': 0.17593805903884294,
  'logloss': 0.52639625476901,
  'auc': 0.8179033844758462},
 'dtc': {'brier': 0.16095,
  'logloss': 5.801226012978405,
  'auc': 0.8390519847629963}}

## Calibrando classificadores usando o Sci-kit Learn
A calibração de classificadores no Sci-kit Learn é implantada pela classe [CalibratedClassifierCV](https://scikit-learn.org/stable/modules/generated/sklearn.calibration.CalibratedClassifierCV.html#sklearn-calibration-calibratedclassifiercv). Essa classe por padrão faz cross-validation e estima uma série de estimadores para então calibrar. Não é isso que queremos. Queremos treinar um único estimador com *todos* os dados e então calibrar esse classificador. Para isso, simplesmente passamos o parâmetro `ensemble=False`.

### Opção 1: Platt scaling
A primeira opção de calibração é chamada [Platt scaling](https://en.wikipedia.org/wiki/Platt_scaling). Esse método consiste em rodar uma regressão logística com as previsões do modelo, calibrando a probabilidade no processo.

In [7]:
def make_calibrated_classifier(model, **kwargs):
    return calibration.CalibratedClassifierCV(
        model,
        # partições de cross-validation
        cv=5,
        # apenas calibrar por validação cruzada
        ensemble=False,
        # usa todas as CPUs do computador para ser mais rápido
        n_jobs=-1,
        **kwargs
    )

# um partial é uma função que recebe um ou mais argumentos fixos
platt_scaling = functools.partial(make_calibrated_classifier, method="sigmoid")

In [8]:
calibrated_models_platt = {
    model_name: platt_scaling(model).fit(X_train, y_train)
    for model_name, model in models.items()
}

In [9]:
performance_platt = {
    model_name: {
        "brier": metrics.brier_score_loss(y_test, model.predict_proba(X_test)[:, 1]),
        "logloss": metrics.log_loss(y_test, model.predict_proba(X_test)[:, 1]),
        # para mostrar que a calibração não altera a eficácia preditiva do modelo
        "auc": metrics.roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    }
    for model_name, model in calibrated_models_platt.items()
}
performance_platt

{'logreg': {'brier': 0.17329114858312886,
  'logloss': 0.5207803549972551,
  'auc': 0.8191846947961736},
 'naive_bayes': {'brier': 0.176030373460807,
  'logloss': 0.5269003783488727,
  'auc': 0.8179033844758462},
 'dtc': {'brier': 0.13542727330446255,
  'logloss': 0.44218368722127344,
  'auc': 0.8385030096257524}}

### Opção 2: regressão isotônica
Outra opção é estimar uma [Regressão isotônica](https://en.wikipedia.org/wiki/Isotonic_regression) com as previsões do modelo. Mas **atenção**: esse método não é paramétrico, ou seja, precisa mais dados para não gerar overfitting. O Sci-kit Learn recomenda pelo menos 1000 linhas.

In [10]:
isotonic_scaling = functools.partial(make_calibrated_classifier, method="isotonic")

In [11]:
calibrated_models_isotonic = {
    model_name: isotonic_scaling(model).fit(X_train, y_train)
    for model_name, model in models.items()
}

In [12]:
performance_isotonic = {
    model_name: {
        "brier": metrics.brier_score_loss(y_test, model.predict_proba(X_test)[:, 1]),
        "logloss": metrics.log_loss(y_test, model.predict_proba(X_test)[:, 1]),
        # para mostrar que a calibração não altera a eficácia preditiva do modelo
        "auc": metrics.roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    }
    for model_name, model in calibrated_models_platt.items()
}
performance_isotonic

{'logreg': {'brier': 0.17329114858312886,
  'logloss': 0.5207803549972551,
  'auc': 0.8191846947961736},
 'naive_bayes': {'brier': 0.176030373460807,
  'logloss': 0.5269003783488727,
  'auc': 0.8179033844758462},
 'dtc': {'brier': 0.13542727330446255,
  'logloss': 0.44218368722127344,
  'auc': 0.8385030096257524}}

## Resultados

In [13]:
pd.concat([
    pd.DataFrame.from_dict(performance, orient="index").assign(method="uncalibrated"),
    pd.DataFrame.from_dict(performance_platt, orient="index").assign(method="platt"),
    pd.DataFrame.from_dict(performance_isotonic, orient="index").assign(method="isotonic"),
]).reset_index().rename(columns={"index": "model"}).set_index(["method", "model"])

brier   logloss       auc
method       model                                    
uncalibrated logreg       0.173290  0.520779  0.819185
             naive_bayes  0.175938  0.526396  0.817903
             dtc          0.160950  5.801226  0.839052
platt        logreg       0.173291  0.520780  0.819185
             naive_bayes  0.176030  0.526900  0.817903
             dtc          0.135427  0.442184  0.838503
isotonic     logreg       0.173291  0.520780  0.819185
             naive_bayes  0.176030  0.526900  0.817903
             dtc          0.135427  0.442184  0.838503